In [ ]:
# source: https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


## original harmony

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
rng = np.random.RandomState(88)
def PCA_omics(source_df, n_pc, visit_dict, metadata = None):
    # zero mean for PCA
    source_df = source_df-source_df.mean()
    # fit PCA
    pca_results  = PCA(n_components = n_pc, random_state=rng)
    pca_results.fit(source_df)
    variances = pca_results.explained_variance_ratio_
    # ecdf
    ecdf_var = [variances[:i].sum() for i in range(len(variances))]
    fig, ax1 = plt.subplots(1,1,figsize = (20, 8))
    ax1.plot(range(len(ecdf_var)),ecdf_var)
    plt.xticks(range(len(ecdf_var)), range(len(ecdf_var)))
    plt.yticks([0.1*i for i in range(11)])
    plt.show()
    plt.savefig("ola_PCs.png")
    # pca annotation and visualization
    transformed = pca_results.transform(source_df)
    df = pd.DataFrame([i.split('.') for i in source_df.index])
    df.columns = ["Patient", "Day"]
    df["PID"] = source_df.index
    for i in range(n_pc):
        df[f"PC{i}"] = transformed[:, i] 
    dat_l = [visit_dict[sample] for sample in df['Patient']]
    df['Day'] = list(dat_l)
    
    try:
        with_sex_df = metadata[metadata["Sam_id"].isin(df["Patient"])]
        with_sex_df = with_sex_df.dropna(subset = ["Sex"])
        df = df.set_index('PID').join(with_sex_df.set_index('Sam_id')) 
    except:
        print("no metadata used")
    
    df['Dataset'] = ['New' if ('193' in patient) else 'Old' for patient in list(df['Patient'])]
    fig, ax =  plt.subplots(1, 1, sharex=True, sharey=True, figsize=(20, 10))
    #df = df.set_index('PID').join(with_ttp_df.set_index('PID_Visit'))
    sns.scatterplot(df["PC0"] , df["PC1"], hue = df["Day"], style = df["Dataset"], s = 100,ax = ax, cmap = "tab10", x_jitter = True)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
 
    # TODO: add more for each category
    fig, ax =  plt.subplots(1, 1, sharex=True, sharey=True, figsize=(20, 10))
    # ax1.plot(range(len(ecdf_var)),ecdf_var
    sns.scatterplot(df["PC0"] , df["PC1"], hue = df["Day"], style = df["Dataset"], s = 100,ax = ax, cmap = "tab10", x_jitter = True)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    ax.set(xlabel='PC0, explained '+str(round(variances[0], 5)*100)+"%", ylabel='PC1, explained '+str(round(variances[1], 5)*100)+"%")
    colors = ["orange", "blue", "red", "green"]
    for index, timepoint in enumerate(["Day_0", "Wk_2", "Mo_2", "Mo_6"]):
    # for index, timepoint in enumerate(["Day_0", "Wk_2", ]):
        currentDF = df.loc[df['Day'] == timepoint]
        confidence_ellipse(currentDF['PC0'], currentDF['PC1'], ax, n_std=1.5, edgecolor=colors[index])
    plt.savefig("jiligala_PCs.png")
    ## add contours for two batches
    fig, ax =  plt.subplots(1, 1, sharex=True, sharey=True, figsize=(20, 10))
    # ax1.plot(range(len(ecdf_var)),ecdf_var)

    ls_l = ["dotted", "--"]
    sns.scatterplot(df["PC0"] , df["PC1"], hue = df["Day"], style = df["Dataset"], s = 100,ax = ax, cmap = "tab10", x_jitter = True)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    ax.set(xlabel='PC0, explained '+str(round(variances[0], 5)*100)+"%", ylabel='PC1, explained '+str(round(variances[1], 5)*100)+"%")
    for index, batch in enumerate(["New", 'Old']):
        currentDF = df.loc[df['Dataset'] == batch]
        confidence_ellipse(currentDF['PC0'], currentDF['PC1'], ax, n_std=1.5, edgecolor="grey", ls=ls_l[index])

    plt.savefig("hehe_PCs.png")
    return df        

In [ ]:
data_mat = pd.read_csv("/home/fuc/HRZE_TB/tom_organized_codes/batch_correction_PCA/1021_microbiome_batchcorrection/microbiome_merged_intersect_1023.csv", index_col="Unnamed: 0")
data_mat = np.array(data_mat)

# zero mean for PCA
data_mat = data_mat-data_mat.mean()
# fit PCA
pca_results  = PCA(n_components = 30, random_state=rng)
pca_results.fit(data_mat)
data_mat = pca_results.transform(data_mat).T
data_mat = pd.DataFrame(data_mat, columns=meta_data.PID_Visit)
meta_data = pd.read_csv("/home/fuc/HRZE_TB/tom_organized_codes/batch_correction_PCA/1021_microbiome_batchcorrection/intersect_metadata_1023.csv")
vars_use = ["Dataset", "Sex"]
# from harmony_copy import run_harmony
# from harmony import run_harmony
from harmony_old import run_harmony
ho = run_harmony(data_mat, meta_data, vars_use)
res = pd.DataFrame(ho.Z_corr)
source_df = res
source_df = source_df.T


# establish dttp and visit dicts
Sam_id_metaorder = [sam.split(".")[0] for sam in meta_data.PID]
visit_dict = dict(zip(Sam_id_metaorder, meta_data.Visit))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = source_df.T.to_numpy()
scaler.fit(data)
ss_scaled = scaler.transform(data).T
ss_source_df = pd.DataFrame(ss_scaled, columns = source_df.columns, index=meta_data.PID_Visit)
df = PCA_omics(ss_source_df, n_pc=30, visit_dict=visit_dict)

## new harmony

In [ ]:
# source: https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
rng = np.random.RandomState(88)
def PCA_omics(source_df, n_pc, visit_dict, metadata = None):
    # zero mean for PCA
    source_df = source_df-source_df.mean()
    # fit PCA
    pca_results  = PCA(n_components = n_pc, random_state=rng)
    pca_results.fit(source_df)
    variances = pca_results.explained_variance_ratio_
    # ecdf
    ecdf_var = [variances[:i].sum() for i in range(len(variances))]
    fig, ax1 = plt.subplots(1,1,figsize = (20, 8))
    ax1.plot(range(len(ecdf_var)),ecdf_var)
    plt.xticks(range(len(ecdf_var)), range(len(ecdf_var)))
    plt.yticks([0.1*i for i in range(11)])
    plt.show()
    plt.savefig("ola_1.png")
    # pca annotation and visualization
    transformed = pca_results.transform(source_df)
    df = pd.DataFrame([i.split('.') for i in source_df.index])
    df.columns = ["Patient", "Day"]
    df["PID"] = source_df.index
    for i in range(n_pc):
        df[f"PC{i}"] = transformed[:, i] 
    dat_l = [visit_dict[sample] for sample in df['Patient']]
    df['Day'] = list(dat_l)
    
    try:
        with_sex_df = metadata[metadata["Sam_id"].isin(df["Patient"])]
        with_sex_df = with_sex_df.dropna(subset = ["Sex"])
        df = df.set_index('PID').join(with_sex_df.set_index('Sam_id')) 
    except:
        print("no metadata used")
    
    df['Dataset'] = ['New' if ('193' in patient) else 'Old' for patient in list(df['Patient'])]
    fig, ax =  plt.subplots(1, 1, sharex=True, sharey=True, figsize=(20, 10))
    #df = df.set_index('PID').join(with_ttp_df.set_index('PID_Visit'))
    sns.scatterplot(df["PC0"] , df["PC1"], hue = df["Day"], style = df["Dataset"], s = 100,ax = ax, cmap = "tab10", x_jitter = True)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
 
    # TODO: add more for each category
    fig, ax =  plt.subplots(1, 1, sharex=True, sharey=True, figsize=(20, 10))
    # ax1.plot(range(len(ecdf_var)),ecdf_var
    sns.scatterplot(df["PC0"] , df["PC1"], hue = df["Day"], style = df["Dataset"], s = 100,ax = ax, cmap = "tab10", x_jitter = True)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    ax.set(xlabel='PC0, explained '+str(round(variances[0], 5)*100)+"%", ylabel='PC1, explained '+str(round(variances[1], 5)*100)+"%")
    colors = ["orange", "blue", "red", "green"]
    for index, timepoint in enumerate(["Day_0", "Wk_2", "Mo_2", "Mo_6"]):
    # for index, timepoint in enumerate(["Day_0", "Wk_2", ]):
        currentDF = df.loc[df['Day'] == timepoint]
        confidence_ellipse(currentDF['PC0'], currentDF['PC1'], ax, n_std=1.5, edgecolor=colors[index])
    plt.savefig("jiligala_1.png")
    ## add contours for two batches
    fig, ax =  plt.subplots(1, 1, sharex=True, sharey=True, figsize=(20, 10))
    # ax1.plot(range(len(ecdf_var)),ecdf_var)

    ls_l = ["dotted", "--"]
    sns.scatterplot(df["PC0"] , df["PC1"], hue = df["Day"], style = df["Dataset"], s = 100,ax = ax, cmap = "tab10", x_jitter = True)
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    ax.set(xlabel='PC0, explained '+str(round(variances[0], 5)*100)+"%", ylabel='PC1, explained '+str(round(variances[1], 5)*100)+"%")
    for index, batch in enumerate(["New", 'Old']):
        currentDF = df.loc[df['Dataset'] == batch]
        confidence_ellipse(currentDF['PC0'], currentDF['PC1'], ax, n_std=1.5, edgecolor="grey", ls=ls_l[index])

    plt.savefig("hehe_1.png")
    return df        

In [ ]:
import numpy as np
import pandas as pd
data_mat = pd.read_csv("/home/fuc/HRZE_TB/tom_organized_codes/batch_correction_PCA/1021_microbiome_batchcorrection/microbiome_merged_intersect_1023.csv", index_col="Unnamed: 0")
data_mat = np.array(data_mat)
meta_data = pd.read_csv("/home/fuc/HRZE_TB/tom_organized_codes/batch_correction_PCA/1021_microbiome_batchcorrection/intersect_metadata_1023.csv")
vars_use = ["Dataset", "Sex"]
from harmony_copy import run_harmonicMic
# from harmony import run_harmony
# from harmony_old import run_harmony
ho = run_harmonicMic(data_mat, meta_data, vars_use)
res = pd.DataFrame(ho.Z_corr)
source_df = res
a = pd.read_csv("/home/fuc/HRZE_TB/tom_organized_codes/batch_correction_PCA/1021_microbiome_batchcorrection/microbiome_merged_intersect_1023.csv", index_col="Unnamed: 0")
source_df.index = a.columns
source_df = source_df.T


# establish dttp and visit dicts
Sam_id_metaorder = [sam.split(".")[0] for sam in meta_data.PID]
visit_dict = dict(zip(Sam_id_metaorder, meta_data.Visit))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = source_df.T.to_numpy()
scaler.fit(data)
ss_scaled = scaler.transform(data).T
ss_source_df = pd.DataFrame(ss_scaled, columns = source_df.columns, index=meta_data.PID_Visit)
df = PCA_omics(ss_source_df, n_pc=30, visit_dict=visit_dict)